# Frequency tests for posts 
This notebook is the analysis for identifying common and rare words within and outside of r/science (Table 2). 
This includes for first time and recurrent posters in r/science.

In [9]:
import re
import csv
import json
import string
from collections import Counter

import spacy
import numpy as np
import pandas as pd
from scipy import stats


import os 
os.chdir('/homes/gws/taugust/Projects/ARK/community_guidelines/')

all_posts_dir = 'data/cleaned/real_subs_cleaned_posts_2018.csv'
# science_related_posts_dir = 'data/cleaned/real_subs_cleaned_posts_2018_science_related.csv'

In [10]:
# define a quick and simple tokenizer
# (FWIW: I'm pretty sure I created this for something else, it's not perfect but ...
# ... the point is to remove punctuation somewhat sensibly, lower case, and split)

punct_chars = list(set(string.punctuation) - set("'"))
punct_chars.sort()
punctuation = ''.join(punct_chars)
replace = re.compile('[%s]' % re.escape(punctuation))

def text_to_tokens(text, lower=True, ngram=None):
    # replace underscores with spaces
    text = re.sub(r'_', ' ', text)
    # break off single quotes at the ends of words (e.g. 'test' -> test)
    text = re.sub(r'\s\'', ' ', text)
    text = re.sub(r'\'\s', ' ', text)
    # remove periods (e.g. U.S. -> US)
    text = re.sub(r'\.', '', text)
    # replace all other punctuation (except single quotes) with spaces (e.g. T-rex -> t rex)
    text = replace.sub(' ', text)
    # remove single quotes (e.g. don't -> dont)
    text = re.sub(r'\'', '', text)
    # replace all whitespace with a single space
    text = re.sub(r'\s', ' ', text)
    # strip off spaces on either end
    text = text.strip()    
    if lower:
        text = text.lower()
    split_text = text.split()
    if ngram is None:
        return split_text
    else:
        return [tuple(split_text[i:i+ngram]) for i in range(len(split_text)-ngram+1)]
        

In [11]:
# read in all posts as a pandas object
# infile = '../../../zinhome/dallas/reddit/all_posts_2018.csv'
df = pd.read_csv(all_posts_dir, index_col=None, header=0, quoting=csv.QUOTE_ALL, escapechar='\\')
print(df.head())
print(df.shape)

   created_utc   subreddit        author           domain  \
0   1532224574  askscience     mstilw577  self.askscience   
1   1532224786  askscience  ballsosteele  self.askscience   
2   1532224926  askscience       ntb2022  self.askscience   
3   1532225571  askscience       tbqh123  self.askscience   
4   1532229925  askscience    Ectricious  self.askscience   

                                                 url  num_comments  score  \
0  https://www.reddit.com/r/askscience/comments/9...             5      2   
1  https://www.reddit.com/r/askscience/comments/9...            13      3   
2  https://www.reddit.com/r/askscience/comments/9...             3      4   
3  https://www.reddit.com/r/askscience/comments/9...            10      2   
4  https://www.reddit.com/r/askscience/comments/9...             6      1   

   ups  downs                                              title  ...  \
0  NaN    NaN                 How do animals get stuck in amber?  ...   
1  NaN    NaN  Why does 

In [12]:
# df_dropped = df.drop(314793).drop(378189) # only two that are getting in the way

In [13]:
# convert to a list of json objects (THIS WILL TAKE A WHILE)
outlines = []
for i in df.index:
    row = df.iloc[i]
    outlines.append({'id': 'p' + str(i), 'subreddit': row['subreddit'], 'author': str(row['author']), 'title': str(row['title']), 'selftext': str(row['selftext']), 'month': int(row['created_month'])})

print(len(outlines)) 


115051


In [14]:
# tokenize each instance (THIS WILL ALSO TAKE A WHILE)
for line in outlines:
    line['title_tokens'] = text_to_tokens(line['title'], ngram=1)
    line['selftext_tokens'] = text_to_tokens(line['selftext'], ngram=1)

In [15]:
print(outlines[0])

{'id': 'p0', 'subreddit': 'askscience', 'author': 'mstilw577', 'title': 'How do animals get stuck in amber?', 'selftext': 'I was wondering how animals and insects get stuck in amber. Did the animals fall into the amber before it solidified? Or does it happen differently?', 'month': 7, 'title_tokens': [('how',), ('do',), ('animals',), ('get',), ('stuck',), ('in',), ('amber',)], 'selftext_tokens': [('i',), ('was',), ('wondering',), ('how',), ('animals',), ('and',), ('insects',), ('get',), ('stuck',), ('in',), ('amber',), ('did',), ('the',), ('animals',), ('fall',), ('into',), ('the',), ('amber',), ('before',), ('it',), ('solidified',), ('or',), ('does',), ('it',), ('happen',), ('differently',)]}


In [16]:
# write the list of jsob objects to a file (just in case you want to re-start this analysis later)
with open('data/cleaned/all_posts_2018.jsonlist', 'w') as f:
    for line in outlines:
        f.write(json.dumps(line) + '\n')

In [17]:
# read it back in (START HERE if you have already done the above and are re-starting this notebook)
with open('data/cleaned/all_posts_2018.jsonlist') as f:
    posts = f.readlines()
posts = [json.loads(line) for line in posts]          

In [18]:
# count the number of posts in each subreddit
subreddit_counter = Counter()
subreddit_counter.update([line['subreddit'] for line in posts])
print(subreddit_counter.most_common())

[('AskHistorians', 41203), ('science', 22157), ('Futurology', 20127), ('askscience', 12162), ('dataisbeautiful', 8437), ('EverythingScience', 6772), ('TrueReddit', 4193)]


In [5]:
# split into science and non-science
# making this instead askscience
science = [line for line in posts if line['subreddit'] == 'science']
print(len(science))
background = [line for line in posts if line['subreddit'] != 'science']
print(len(background))

6772
108279


In [6]:
count = 0
word = 'bàn'
for post in science:
    if word in post['title_tokens'] or word in post['selftext_tokens']:
        print(post)
        count += 1
    if count > 10:
        break

In [7]:
# convert list for bigrams to tuple
def convert_to_tuple(line, cols):
    for col in cols:
        line[col] = [tuple(bigram) for bigram in line[col]]
    return line

science = list(map(lambda line: convert_to_tuple(line, ['selftext_tokens', 'title_tokens']), science))
background = list(map(lambda line: convert_to_tuple(line, ['selftext_tokens', 'title_tokens']), background))


In [8]:
# count the tokens in science and background
science_counts = Counter()
bg_counts = Counter()

# *** modify these to choose whether to use title and/or body ***
use_title = True
use_selftext = True

for line in science:
    if use_title:
        science_counts.update(line['title_tokens'])
    if use_selftext:
        science_counts.update(line['selftext_tokens'])
for line in background:
    if use_title:
        bg_counts.update(line['title_tokens'])    
    if use_selftext:
        bg_counts.update(line['selftext_tokens'])    
print(len(science_counts), len(bg_counts))

# make a common vocabulary
common = set()
common.update(science_counts.keys())
common.update(bg_counts.keys())
common = list(common)
common.sort()
print(len(common))

# get the the total number of tokens in each
science_N = sum(science_counts.values())
bg_N = sum(bg_counts.values())
print(science_N, bg_N)

16274 112285
114320
122653 5127351


In [9]:
# convert counts to arrays of size equal to the vocabulary (common)
science_counts_array = np.array([science_counts[w] if w in science_counts else 0 for w in common])
bg_counts_array = np.array([bg_counts[w] if w in bg_counts else 0 for w in common])

In [10]:
# do a Chi-squared test on the difference in counts (just for fun)
from scipy import stats
print(stats.chisquare(science_counts_array, bg_counts_array+0.001))

Power_divergenceResult(statistic=14473359.836986423, pvalue=0.0)


In [11]:
"""
smoothing = 0.1
bg_counts_array = bg_counts_array + smoothing
bg_counts_N_temp = bg_counts_array.sum()
pvals = []
testvals = []
for i in range(len(common)):
    chisq, p = stats.chisquare([science_counts_array[i], science_N - science_counts_array[i]], 
                              [bg_counts_array[i], bg_counts_N_temp - bg_counts_array[i]])
    pvals.append(p)
    testvals.append(chisq)
"""    

'\nsmoothing = 0.1\nbg_counts_array = bg_counts_array + smoothing\nbg_counts_N_temp = bg_counts_array.sum()\npvals = []\ntestvals = []\nfor i in range(len(common)):\n    chisq, p = stats.chisquare([science_counts_array[i], science_N - science_counts_array[i]], \n                              [bg_counts_array[i], bg_counts_N_temp - bg_counts_array[i]])\n    pvals.append(p)\n    testvals.append(chisq)\n'

In [12]:
# compute the probability of each word count in science, using probabilities estimated from the background
smoothing = 0.1
probs = []
pvals = []
vocab_size = len(common)
for word in common:
    # convert counts in the background into (smoothed) probabilities
    if word in bg_counts:
        bg_c = bg_counts[word] + smoothing
        bg_p = (bg_counts[word] + smoothing) / float(bg_N + smoothing * len(common))
    else:
        bg_c = smoothing
        bg_p = (smoothing) / float(bg_N + smoothing * len(common))
    # get the word counts in science
    if word in science_counts:
        count = science_counts[word]
    else:
        count = 0
    # for each word, compute the probability of the observed count, given the background frequency
    prob = stats.binom.logpmf(k=count, n=science_N, p=bg_p)
    # also do a chi-squared test
    chisq, pval = stats.chisquare([count, science_N - count], 
                              [bg_c, bg_N + smoothing * vocab_size - bg_c])
    # save the pvals and probabilities
    pvals.append(pval * vocab_size)    
    probs.append(prob)
    
print(len(probs))

114320


In [13]:
# sort by the probabilities computed above and display the top terms
order = list(np.argsort(probs))
df = pd.DataFrame(columns=['word', 'probability', 'pval', 'science_freq', 'bg_freq'])
for i in range(1, 50000):
    # print those words that have *higher* probability in science than background
    word = common[order[i]]
    if word in science_counts:
        sc_count = science_counts[word]
    else:
        sc_count = 0
    if word in bg_counts:
        bg_count = bg_counts[word]
    else:
        bg_count = 0
    if sc_count / science_N > bg_count / bg_N:
        df.loc[i] = [word, probs[order[i]], pvals[order[i]], sc_count / science_N, bg_count / bg_N]
# actually, sort on multiple values because of round-off error in pvals
df = df.sort_values(['probability', 'science_freq'],  ascending=[True, False])
df.to_csv('common_in_science_posts.csv')
print(df.head(n=60))

                  word  probability  pval  science_freq       bg_freq
2        (scientists,)  -657.339602   0.0      0.004378  5.419953e-04
4               (new,)  -573.230322   0.0      0.007786  2.066369e-03
6             (study,)  -528.386195   0.0      0.005088  9.835488e-04
7           (climate,)  -484.762156   0.0      0.003123  3.695866e-04
8    (scienceseekers,)  -480.348014   0.0      0.000432  0.000000e+00
9      (sciseekpicks,)  -480.348014   0.0      0.000432  0.000000e+00
10          (scicomm,)  -353.283422   0.0      0.000432  1.950325e-07
12              (may,)  -292.919288   0.0      0.003335  7.627720e-04
14              (epa,)  -284.311455   0.0      0.000652  7.216202e-06
15         (research,)  -255.464852   0.0      0.002976  6.960709e-04
16      (researchers,)  -252.705769   0.0      0.002479  4.914819e-04
17                (–,)  -252.345643   0.0      0.001786  2.422303e-04
19                (и,)  -236.323031   0.0      0.000440  2.145357e-06
20           (cancer

In [14]:
# do the same thing, but for those words that have  *lower* probability in science than background
order = list(np.argsort(probs))
df = pd.DataFrame(columns=['word', 'probability', 'pval', 'science_freq', 'bg_freq'])
for i in range(50000):
    word = common[order[i]]
    if word in science_counts:
        sc_count = science_counts[word]
    else:
        sc_count = 0
    if word in bg_counts:
        bg_count = bg_counts[word]
    else:
        bg_count = 0
    if sc_count / science_N < bg_count / bg_N:
        df.loc[i] = [word, probs[order[i]], pvals[order[i]], sc_count / science_N, bg_count / bg_N]
df = df.sort_values(['probability', 'science_freq'],  ascending=[True, False])
df.to_csv('rare_in_science.csv')
print(df.head(n=60))

              word  probability  pval  science_freq   bg_freq
1             (i,)  -882.999538   0.0      0.000848  0.010099
3           (was,)  -593.647384   0.0      0.001337  0.008632
5           (did,)  -553.670344   0.0      0.000351  0.005824
11           (or,)  -302.223886   0.0      0.001794  0.006551
13         (were,)  -291.752873   0.0      0.000946  0.004843
18          (any,)  -241.750982   0.0      0.000334  0.003021
21        (there,)  -228.659420   0.0      0.000905  0.004115
22          (the,)  -221.871074   0.0      0.040341  0.053252
23        (would,)  -220.772163   0.0      0.000823  0.003875
25         (what,)  -207.980370   0.0      0.002389  0.006414
27           (it,)  -199.940396   0.0      0.004183  0.008970
29           (im,)  -195.104376   0.0      0.000065  0.001861
31           (if,)  -181.457598   0.0      0.001150  0.004052
33         (this,)  -173.268451   0.0      0.002788  0.006554
34         (they,)  -172.093404   0.0      0.001622  0.004737
35      

In [15]:
# count post authors
post_authors = Counter()
post_authors.update([line['author'] for line in science])
np.histogram([v for v in post_authors.values()], bins = np.arange(20))

(array([  0, 948, 159,  73,  36,  26,  28,  18,   9,  16,   8,   4,   4,
          8,   6,   8,   8,   1,   4]),
 array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
        17, 18, 19]))

In [16]:
# count rare vs common posters
new_posters = [line for line in science if post_authors[line['author']] == 1]
regular_posters = [line for line in science if post_authors[line['author']] >= 5]
print(len(new_posters))
print(len(regular_posters))

948
5143


In [17]:
# do the same sort of comparison comparing rare vs frequent posters
newuser_counts = Counter()
regular_counts = Counter()
use_title = True
use_selftext = True
for line in new_posters:
    # added check that title doesn't have ama (jama is a journal)
#     if (any(ama_str in line['title'] for ama_str in ['AMA', 'this ama'])) and (not 'JAMA' in line['title']):
#         continue
    if use_title:
        newuser_counts.update(line['title_tokens'])
    if use_selftext:
        newuser_counts.update(line['selftext_tokens'])
for line in regular_posters:
    if use_title:
        regular_counts.update(line['title_tokens'])    
    if use_selftext:
        regular_counts.update(line['selftext_tokens'])    
print(len(newuser_counts), len(regular_counts))
common = set()
common.update(newuser_counts.keys())
common.update(regular_counts.keys())
common = list(common)
common.sort()
print(len(common))
new_N = sum(newuser_counts.values())
reg_N = sum(regular_counts.values())
print(new_N, reg_N)

4977 14040
15433
16086 95132


In [18]:
smoothing = 0.1
probs = []
pvals = []
vocab_size = len(common)
for word in common:
    if word in regular_counts:
        bg_c = regular_counts[word] + smoothing
        bg_p = (regular_counts[word] + smoothing) / float(reg_N + smoothing * len(common))
    else:
        bg_c = smoothing
        bg_p = (smoothing) / float(reg_N + smoothing * len(common))
    if word in newuser_counts:
        count = newuser_counts[word]
    else:
        count = 0
    prob = stats.binom.logpmf(k=count, n=new_N, p=bg_p)
    chisq, pval = stats.chisquare([count, new_N - count], 
                              [bg_c, reg_N + smoothing * vocab_size - bg_c])
    pvals.append(pval * vocab_size)    
    probs.append(prob)
    
print(len(probs))

15433


In [19]:
order = list(np.argsort(probs))
df = pd.DataFrame(columns=['word', 'probability', 'pval', 'science_freq', 'bg_freq'])
for i in range(1, 20000):
    word = common[order[i]]
    if word in newuser_counts:
        sc_count = newuser_counts[word]
    else:
        sc_count = 0
    if word in regular_counts:
        bg_count = regular_counts[word]
    else:
        bg_count = 0
    if sc_count / new_N > bg_count / reg_N:
        df.loc[i] = [word, probs[order[i]], pvals[order[i]], sc_count / new_N, bg_count / reg_N]
df = df.sort_values(['probability', 'science_freq'],  ascending=[True, False])
df.to_csv('common_in_science_posts_newusers.csv')
print(df.head(n=60))

IndexError: list index out of range

In [ ]:
order = list(np.argsort(probs))
df = pd.DataFrame(columns=['word', 'probability', 'pval', 'science_freq', 'bg_freq'])
for i in range(1, 20000):
    word = common[order[i]]
    if word in newuser_counts:
        sc_count = newuser_counts[word]
    else:
        sc_count = 0
    if word in regular_counts:
        bg_count = regular_counts[word]
    else:
        bg_count = 0
    if sc_count / new_N < bg_count / reg_N:
        df.loc[i] = [word, probs[order[i]], pvals[order[i]], sc_count / new_N, bg_count / reg_N]
df = df.sort_values(['probability', 'science_freq'],  ascending=[True, False])
df.to_csv('rare_in_science_posts_newusers.csv')
print(df.head(n=60))

In [40]:
count = 0
word = 'bàn'
for post in science:
    if word in post['title_tokens'] or word in post['selftext_tokens']:
        print(post)
        count += 1
    if count > 10:
        break